In [3]:
from AirSimClient import *
import numpy as np
import pandas as pd
import PIL
from skimage.transform import resize
from time import sleep
import pickle
from helpers import nearPos

In [4]:
car_client = CarClient()
car_client.confirmConnection()
car_controls = CarControls()
print('Connection established!')

Waiting for connection: 
Connection established!


In [8]:
# convert all the segments to one color
found = car_client.simSetSegmentationObjectID("[\w]*", 15, True)
# convert one of the lanes to a different color
found = car_client.simSetSegmentationObjectID("Landscape13", 19)
print("Done: %r" % (found))

Done: True


In [12]:
def get_label(car_client, i):
        image_response = car_client.simGetImages([ImageRequest(0, AirSimImageType.Segmentation, False, False)])[0]
        image1d = np.fromstring(image_response.image_data_uint8, dtype=np.uint8)
        image_rgba = image1d.reshape(image_response.height, image_response.width, 4)
 
        #Remove alpha channel
        image_rgba = image_rgba[32:132, 0:255, 0:3]
        
        # remove all the green background color to leave only the label
        image_rgba[np.where((image_rgba == [68,218,116]).all(axis = 2))] = [0,0,0]
        
        image_pil = PIL.Image.fromarray(image_rgba)
        
        image_pil.save('images/labelled/out_%d.jpg' % i)
        np.save('images/labelled/out_%d.npy' % i, image_rgba)

        # resize the images to a smaller size so that they're easier to label and convert them to grayscale
        
        image_pil = PIL.Image.fromarray(image_rgba).convert('L').resize((160, 80), resample=PIL.Image.LANCZOS)
        image_data = np.asarray(image_pil.getdata(), dtype=float) / 255.0
        
        np.save('images/labelled/out_1d_%d.npy' % i, image_data.reshape((80, 160, 1)))
        
def get_scene(car_client, i, TEST=False):
        image_response = car_client.simGetImages([ImageRequest(0, AirSimImageType.Scene, False, False)])[0]
        image1d = np.fromstring(image_response.image_data_uint8, dtype=np.uint8)
        image_rgba = image1d.reshape(image_response.height, image_response.width, 4)
 
        #Remove alpha channel
        image_rgba = image_rgba[32:132, 0:255, 0:3]
        
        image_pil = PIL.Image.fromarray(image_rgba)
        if TEST:
            image_pil.save('images/test/out_scene_%d.jpg' % i)
        else:
        
            image_pil.save('images/scene/out_scene_%d.jpg' % i)
            np.save('images/scene/out_scene_%d.npy' % i, image_rgba)

            # resize the images
            image_pil = image_pil.resize((160, 80), resample=PIL.Image.LANCZOS)
            image_data = np.asarray(image_pil.getdata(), dtype=float)

            np.save('images/scene/out_reshaped_scene_%d.npy' % i, image_data.reshape((80, 160, 3)))
                    

In [ ]:
# collect data and save the images both as images and numpy arrays
# set location for collecting training data
# frame rate is dependent on your particular computer configuration. With an i9 processor and dual-GeForce GTX 1080 TI, 30 FPS is achievable.

car_client.simSetPose({b'position': {b'x_val': 858.1627197265625,
  b'y_val': -397.84429931640625,
  b'z_val': 0.1847546398639679},
 b'orientation': {b'w_val': 0.9997275471687317,
  b'x_val': -0.00038991699693724513,
  b'y_val': -0.00528569333255291,
  b'z_val': 0.02273230068385601}}, True)


endTrainPos = {b'position': {b'x_val': 36.63443374633789,
  b'y_val': -17.22163963317871,
  b'z_val': 0.1286291480064392},
 b'orientation': {b'w_val': 0.26637503504753113,
  b'x_val': 0.0003908914513885975,
  b'y_val': 7.000972982496023e-05,
  b'z_val': -0.9638693332672119}}

i = 0
while(not nearPos(car_client.simGetPose(), endTrainPos)):
    get_label(car_client, i)
    get_scene(car_client, i)    
    i+=1

C:\Users\msne-garage01\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\msne-garage01\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [13]:
# Collect testing images
# set location for collecting test data
car_client.simSetPose({b'position': {b'x_val': 858.1627197265625,
  b'y_val': -397.84429931640625,
  b'z_val': 0.1847546398639679},
 b'orientation': {b'w_val': 0.9997275471687317,
  b'x_val': -0.00038991699693724513,
  b'y_val': -0.00528569333255291,
  b'z_val': 0.02273230068385601}}, True)

endTestPos = {b'position': {b'x_val': 858.1627197265625,
  b'y_val': -397.84429931640625,
  b'z_val': 0.1847546398639679},
 b'orientation': {b'w_val': 0.9997275471687317,
  b'x_val': -0.00038991699693724513,
  b'y_val': -0.00528569333255291,
  b'z_val': 0.02273230068385601}}

i = 0
while(not nearPos(car_client.simGetPose(), endTestPos)):
    get_scene(car_client, i, TEST=True)
    i+=1

C:\Users\msne-garage01\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


KeyboardInterrupt: 

In [9]:
import numpy as np
import pickle
import PIL

label_lst = []
scene_lst = []

RESIZE = True

start = 0
end = 786

for i in range(start, end):
    
    # If the images aren't properly resized, resize them
    if RESIZE:
        label = np.load('images/labelled/out_%d.npy' % i)
        scene = np.load('images/scene/out_scene_%d.npy' % i)
    
        image_pil = PIL.Image.fromarray(label).convert('L').resize((160, 80), resample=PIL.Image.LANCZOS)
        label = np.asarray(image_pil.getdata(), dtype=float)
        label = label.reshape((80,160,1))

        image_pil = PIL.Image.fromarray(scene).resize((160, 80), resample=PIL.Image.LANCZOS)
        scene = np.asarray(image_pil.getdata(), dtype=float)
        scene = scene.reshape((80,160,3))

        np.save('images/labelled/out_1d_%d.npy' % i, label)

        np.save('images/scene/out_reshaped_scene_%d.npy' % i, scene)
    
    else:
        label = np.load('images/labelled/out_1d_%d.npy' % i)
        scene = np.load('images/scene/out_reshaped_scene_%d.npy' % i)
        print(label)
    
    label_lst.append(label)
    scene_lst.append(scene)
    
pickle.dump(label_lst, open('images/labelled/out_1d_labels.p', 'wb'))
pickle.dump(scene_lst, open('images/scene/out_reshaped_scenes.p', 'wb'))